In [1]:
import numpy as np
import os
import random
import tensorflow as tf

from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Dropout, Flatten, Dense, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

from scipy.io import wavfile
from glob import glob
from python_speech_features import mfcc

In [2]:
input_dir = "speaker"
train_batch_size = 32
val_batch_size = 8
labels = os.listdir(input_dir)

In [3]:
nfilt = 26
nfeat = 13
nfft = 512
rate = 16000
step = rate // 10

In [12]:
def features_from_txt(filename):
    folder = os.path.basename(os.path.dirname(filename))
    rate, wav = wavfile.read(filename)
    print(rate)
    rand_index = np.random.randint(0, wav.shape[0] - step)
    sample = wav[rand_index:rand_index+step]
    X_sample = mfcc(sample, rate, numcep=nfeat, nfilt=nfilt, nfft=nfft).T
    label = to_categorical(labels.index(folder), num_classes=len(labels))
#     print(wav.shape, sample.shape, X_sample.shape, label.shape)
    return (X_sample, label)

In [13]:
features_from_txt("speaker/0016/00003.wav")

16000


(array([[ 17.20212216,  16.18201001,  14.36013758,  14.31932577,
          14.02138442,  15.11083149,  16.51404539,  16.6281437 ,
          16.37586967],
        [  2.21003638,   9.79423227,  14.66857306,  16.7935322 ,
          16.22336886,  -6.07962933, -17.84182769, -12.38856308,
          -1.95439081],
        [-15.42852318,  -9.96046046,   4.73986217,   5.715574  ,
           6.96967526,   4.5895705 ,  -1.40836829,   0.5849926 ,
          -5.16731947],
        [ 13.95063507,  28.15455247,  13.6929731 ,  10.60649096,
          15.47630398,  12.15249717,  -0.56541711,  -2.0362923 ,
          -3.97100303],
        [-20.25070397, -19.81496733,  -5.15873154,  -3.95465475,
          -4.25719681, -15.20619317, -17.64015345, -23.86790106,
         -18.84739228],
        [-41.38303434, -34.02797722, -23.09328708, -27.18243059,
         -32.28009428,  11.0789619 ,   4.9305127 ,  -5.70873505,
         -24.41232356],
        [-35.75638352, -23.83145243, -22.53328586, -21.29460737,
         -2

In [6]:
def generate_data(input_dir, shuffle=False):
    files = glob(os.path.join(input_dir, "*", "*"))
    random.shuffle(files)
    n = len(files)
    def fetch():
        i = 0
        while True:
            if shuffle:
                file = random.choice(files)
            else:
                file = files[i]
                i = (i+1)%n
            yield features_from_txt(file)
    return fetch

In [7]:
def audio_model():
    x1 = Input(shape=(13, 9), name='audio_input')
    x = LSTM(128, return_sequences=True)(x1)
    x = LSTM(64)(x)
    x = Dense(20, activation='softmax',name='audio_output')(x)
    return Model(inputs=x1, outputs=x)

In [8]:
batch_size = 32
dataset = tf.data.Dataset.from_generator(generate_data(input_dir), output_types=(tf.float64, tf.uint8), output_shapes=((13, 9), (20,)))
dataset = dataset.batch(batch_size)
for d in dataset.take(1):
    print(d[0].shape, d[1].shape)

(32, 13, 9) (32, 20)


In [9]:
epochs = 50
lr = 0.0001
save_model_as = "audio_epochs{}_lr{}_batch{}"
optimizer = Adam(lr=lr)
model = audio_model()
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(dataset, epochs=epochs, steps_per_epoch=20)
model.save(save_model_as.format(epochs, lr, batch_size))

Train for 20 steps
Epoch 1/50
20/20 [==============================] - 8s 403ms/step - loss: 3.0006 - accuracy: 0.0625
Epoch 2/50
20/20 [==============================] - 2s 99ms/step - loss: 2.9956 - accuracy: 0.0516
Epoch 3/50
20/20 [==============================] - 2s 99ms/step - loss: 2.9882 - accuracy: 0.0594
Epoch 4/50
20/20 [==============================] - 2s 98ms/step - loss: 2.9662 - accuracy: 0.0859
Epoch 5/50
20/20 [==============================] - 2s 98ms/step - loss: 2.9607 - accuracy: 0.0688
Epoch 6/50
20/20 [==============================] - 2s 98ms/step - loss: 2.9540 - accuracy: 0.0906
Epoch 7/50
20/20 [==============================] - 2s 99ms/step - loss: 2.9466 - accuracy: 0.0891
Epoch 8/50
20/20 [==============================] - 2s 98ms/step - loss: 2.9432 - accuracy: 0.0719
Epoch 9/50
20/20 [==============================] - 2s 98ms/step - loss: 2.9305 - accuracy: 0.0969
Epoch 10/50
20/20 [==============================] - 2s 99ms/step - loss: 2.9303 - accura